In [2]:
import re
import copy

In [21]:
class DBCFile:
    node = []
    allDatas = []
    siganlList = []
    SignalsName = []
    messageName = []
    filename = ""
    # NodesPattern:节点
    NodesPattern = re.compile(r"BU_: (.*)", re.S)
    # MessagePattern：消息
    MessagePattern = re.compile(r"BO_ (.*?) (.*?): (.*?) (.*)", re.S)
    # SignalPattern：信号
    SignalPattern = re.compile('''SG_ (.*?) : (.*?)\|(.*?)@.*? \((.*?),(.*?)\) \[(.*?)\|(.*?)\] "(.*?)" (.*)''', re.S)
    DefaultValue = '''BA_ "GenSig(.*?)" SG_ (\d+) signalname (\d+);'''
        
    def __init__(self,filename):
        self.filename = filename
    
    def read(self):
        with open(self.filename,encoding="gbk") as f:  # 设置文件对象
            self.allDatas = []
            line = f.readline()
            while line:
                """ 匹配出节点 """
                NodesSearched = re.search(self.NodesPattern, line.strip())
                if NodesSearched:
                    self.node = NodesSearched.group(1).split(" ")
                """ 匹配出消息 """
                MessageSearched = re.search(self.MessagePattern, line.strip())
                if MessageSearched:
                    self.siganlList.clear()
                    """如果匹配到了message，则获取到message的相关参数 
                     比如匹配到了NM_Message_ESC_409，则会解析出改message的一些参数构成list对象['1033', 'NM_Message_ESC_409', '8', 'ESC']
                     这四个参数分别是 messgage ID ;message name ; messgae dataLenth ,message sender
                     而且把这个list对象 加在了 siganlList 索引0的位置
                    """
                    Message = list(MessageSearched.groups())
                    self.siganlList.append(Message)
                    """ 只 要 message的名字 messageName 列表中"""
                    self.messageName.append(Message[1])
                    """读取下一行"""
                    line = f.readline()
                    """因为有些message并没有定义signal，所以 下一行还是message"""
                    MessageSearched = re.search(self.MessagePattern, line.strip())
                    SignalSearched = re.search(self.SignalPattern, line.strip())
                    """下一行如果不是message的内容 就一定是signal的内容了"""
                    if not MessageSearched:
                        while SignalSearched:
                            """获取信号的参数追加到siganlList"""
                            signal = list(SignalSearched.groups())
                            self.siganlList.append(signal)
                            """只获取 signal name"""
                            self.SignalsName.append(signal[0])

                            # 再次解析信号，直到这个message下的信号全部解析完毕
                            line = f.readline()
                            SignalSearched = re.search(self.SignalPattern, line.strip())
                   # print(siganlList)
                    c = copy.deepcopy(self.siganlList)
                    self.allDatas.append(c)
                else:
                    line = f.readline()
                    MessageSearched = re.search(self.MessagePattern, line.strip())

In [22]:
filename = "./data/J1939_bms.dbc"
d = DBCFile(filename)

In [37]:
d.read()
print(len(d.allDatas),len(d.messageName),len(d.siganlList),len(d.SignalsName))

21 147 4 756
